In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, explained_variance_score

In [2]:
RC_circuit_db = pd.read_csv('db_completa_RC_Tecnologia.csv')
RC_circuit_db.head()

,time,voltaje_ent,voltaje_cap_5,Resistencia real,Capacitancia Real
0,0,4.93,0.00,53300,100
1,200,4.93,0.00,53300,100
2,400,4.93,0.02,53300,100
3,600,4.93,0.24,53300,100
4,800,4.93,0.43,53300,100


In [3]:
# Funcion de normalizacion
def normalize(x, xmin, xmax):
    return (xmax - xmin)*((x - np.min(x))/(np.max(x) - np.min(x))) + xmin

In [4]:
#X = [tiempo, R, C, Ve]
#y = [Vc]

R = normalize(RC_circuit_db['Resistencia real'], 0.1, 0.8)
C = normalize(RC_circuit_db['Capacitancia Real'], 0.1, 0.8)
t = normalize(RC_circuit_db['time'], 0.1, 0.8)
Ve = RC_circuit_db['voltaje_ent']

# Input variables
X = np.vstack([R, C, t, Ve]).T

# Target
y = RC_circuit_db['voltaje_cap_5']

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.30, random_state = 2002)

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(14000, 4)
(6000, 4)
(14000,)
(6000,)


In [6]:
class ANN():
    def __init__(self, X, Y, n_neuronas):
        self.x = X
        n_entradas = X.shape[1]
        self.IW = np.random.uniform(size = [n_entradas, n_neuronas])
        self.b = np.random.uniform(size = [n_neuronas])
        self.y = Y

    # Funciones de activacion\n",
    def logsig(self, n):
        return (1/(1 + np.exp(-n)))

    def tansig(self, n):
        return (2/(1 + np.exp(-2*n))) - 1

    def relu(self, n):
        return np.maximum(n, 0, n)

    def train(self):
        self.LW = np.dot(np.linalg.pinv(self.logsig(self.x.dot(self.IW) + self.b)), self.y)

    def predict(self, Xdata):
        self.x = Xdata
        return np.dot(self.logsig(self.x.dot(self.IW) + self.b), self.LW)

In [7]:
def regression_results(y_true, y_pred):
    # Regression metrics
    ev = explained_variance_score(y_true, y_pred)
    mae = mean_absolute_error(y_true, y_pred) 
    mse = mean_squared_error(y_true, y_pred) 
    r2 = r2_score(y_true, y_pred)

    print('explained_variance: ', round(ev,4))    
    print('MAE: ', round(mae,4))
    print('MSE: ', round(mse,4))
    print('R²: ', round(r2,4))

In [8]:
# X [k_muestras x n_variables_entrada]
# y [k_muestras]
    
r2 = 0
goal = 0.98
n_neuronas = 1
    
while r2 < goal:
    model = ANN(X_train, y_train, n_neuronas)
    model.train()
    y_train_pred = model.predict(X_train)
    
    r2 = r2_score(y_train, y_train_pred)
    
    n_neuronas += 1
    
print("Best model with %d neurons" % (n_neuronas))

y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)

regression_results(y_train, y_train_pred)
regression_results(y_test, y_test_pred)

#print(\RMSE (train):\, np.sqrt(np.mean( (y_train-y_train_pred)**2) ))
#print(\RMSE (test):\, np.sqrt(np.mean( (y_test-y_test_pred)**2) ))
regression_results(y_test, y_test_pred)

Best model with 30 neurons
explained_variance:  0.9804
MAE:  0.1079
MSE:  0.056
R²:  0.9804
explained_variance:  0.9806
MAE:  0.1059
MSE:  0.0563
R²:  0.9806
explained_variance:  0.9806
MAE:  0.1059
MSE:  0.0563
R²:  0.9806


In [13]:
import numpy as np
from sklearn.metrics import mean_squared_error

# Asegúrate de definir y_train y y_test en lugar de y_true y y_pred_ann
y_true = y_test  # Valores reales
y_pred_ann = y_test_pred  # Predicciones del modelo ANN

# Calcula el Error Cuadrático Medio (MSE)
mse_ann = mean_squared_error(y_true, y_pred_ann)

# Define la tolerancia para el P-I Test
tolerancia = 1.96 * np.sqrt(mse_ann)  # Intervalo de confianza al 95%

# Realiza el P-I Test
p_i_test_ann = np.abs(y_true - y_pred_ann) <= tolerancia

# Calcula la proporción de valores dentro del intervalo
proporcion_ann = np.mean(p_i_test_ann)

print(f"P-I Test Proporción ANN: {proporcion_ann}")


P-I Test Proporción ANN: 0.9828333333333333


In [14]:
import numpy as np
from sklearn.metrics import mean_squared_error
from scipy.stats import linregress

# Asegúrate de definir y_test y y_test_pred como los datos de valores reales y predichos
y_true = y_test  # Valores reales
y_pred_ann = y_test_pred  # Predicciones del modelo ANN

# Calcula el Error Cuadrático Medio (MSE)
mse_ann = mean_squared_error(y_true, y_pred_ann)

# Define la tolerancia para el P-I Test
tolerancia = 1.96 * np.sqrt(mse_ann)  # Intervalo de confianza al 95%

# Realiza el P-I Test
p_i_test_ann = np.abs(y_true - y_pred_ann) <= tolerancia

# Calcula la proporción de valores dentro del intervalo
proporcion_ann = np.mean(p_i_test_ann)

# Calcula la pendiente y el intercepto
slope, intercept, _, _, _ = linregress(y_true, y_pred_ann)

print(f"P-I Test Proporción ANN: {proporcion_ann}")
print(f"Pendiente: {slope}")
print(f"Intercepto: {intercept}")


P-I Test Proporción ANN: 0.9828333333333333
Pendiente: 0.9803601180604439
Intercepto: 0.06108512531286259
